In [21]:
import pandas as pd
import geopandas as gpd
import requests
import folium
from folium import plugins
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
from io import StringIO
import numpy as np
from branca.colormap import linear


In [3]:
url = 'https://data.virginia.gov/dataset/956da7a2-4e67-4885-bb2e-c7c81bd5909b/resource/d573ca07-5c35-44e0-8e94-1cacb922e1f2/download/virginia_city_county_census_boundaries.geojson'
response = requests.get(url)

geo_data = gpd.read_file(StringIO(response.text))


#Create a map centered around Virginia
m = folium.Map(location=[37.4316, -78.6569], zoom_start=7)

#counties of virginia
folium.GeoJson(
    geo_data,
    name="Virginia Counties",
    style_function=lambda feature: {
        "fillColor": "#ffff00",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5 ",
    }
).add_to(m)




folium.LayerControl().add_to(m)
m.save("virginia_counties_map.html")


In [ ]:
house_hold_size = pd.read_excel("C:/Users/Tony/Downloads/household_size.xlsx")

url = 'https://data.virginia.gov/dataset/956da7a2-4e67-4885-bb2e-c7c81bd5909b/resource/d573ca07-5c35-44e0-8e94-1cacb922e1f2/download/virginia_city_county_census_boundaries.geojson'
response = requests.get(url)

# Read the GeoJSON data
geo_data = gpd.read_file(StringIO(response.text))
print(geo_data)

household_dict = house_hold_size.set_index("County")["Avg"].to_dict()

# Create a map centered around Virginia
m = folium.Map(location=[37.4316, -78.6569], zoom_start=7)

# Add GeoJson layer for Virginia counties with household size
folium.GeoJson(
    geo_data,
    name="Average Household Size",
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME'],  # Field to display the county name
        aliases=['County:'],  # Alias for the field
        localize=True,
        style="""
            background-color: white;
            font-size: 14px;
            border: 2px solid black;
            border-radius: 3px;
            padding: 5px;
        """,
        sticky=False
    ),
    style_function=lambda feature: {
        "fillColor": 'lightblue',  # Use a neutral color for the counties
        "color": "black",
        "weight": 1,
    }
).add_to(m)

# Add tooltips for average household size
for idx, row in geo_data.iterrows():  # Iterate through rows of the GeoDataFrame
    county_name = row['NAME']  # Access county name directly
    avg_household_size = household_dict.get(county_name, 0)

    # Update the tooltip to include the average household size
    folium.GeoJson(
        row['geometry'],
        tooltip=f"{county_name}: {avg_household_size}",
        style_function=lambda feature: {
            "fillColor": 'lightgray',
            "color": "black",
            "weight": 1,
        }
    ).add_to(m)

# Save the map to HTML
m.save("avg_house_hold_size.html")
m

     FID STATEFP COUNTYFP  COUNTYNS  GEOID         NAME            NAMELSAD  \
0      1      51      640  01498426  51640        Galax          Galax city   
1      2      51      760  01789073  51760     Richmond       Richmond city   
2      3      51      670  01498428  51670     Hopewell       Hopewell city   
3      4      51      131  01480151  51131  Northampton  Northampton County   
4      5      51      187  01674058  51187       Warren       Warren County   
..   ...     ...      ...       ...    ...          ...                 ...   
129  130      51      197  01501379  51197        Wythe        Wythe County   
130  131      51      027  01497431  51027     Buchanan     Buchanan County   
131  132      51      071  01494551  51071        Giles        Giles County   
132  133      51      117  01500747  51117  Mecklenburg  Mecklenburg County   
133  134      51      077  01501196  51077      Grayson      Grayson County   

    LSAD CLASSFP  MTFCC  ... CBSAFP METDIVFP FUNCST

In [4]:
average_sales_price_2023 = pd.read_excel("C:/Users/Tony/Downloads/sales_prices.xlsx")

# Clean county names in average_sales_price_2023
average_sales_price_2023['county'] = (
    average_sales_price_2023['county']
    .str.replace(" City", "", regex=False)  # Remove " City"
    .str.replace(" County", "", regex=False)  # Remove " County"
    .str.strip()  # Strip leading and trailing spaces
    .str.lower()  # Convert to lowercase for uniformity
)

# Load geographic data from the URL
url = 'https://data.virginia.gov/dataset/956da7a2-4e67-4885-bb2e-c7c81bd5909b/resource/d573ca07-5c35-44e0-8e94-1cacb922e1f2/download/virginia_city_county_census_boundaries.geojson'
response = requests.get(url)

# Read the GeoJSON data
geo_data = gpd.read_file(StringIO(response.text))

# Clean county names in geo_data
geo_data['NAME'] = (
    geo_data['NAME']
    .str.replace(" City", "", regex=False)  # Remove " City"
    .str.replace(" County", "", regex=False)  # Remove " County"
    .str.strip()  # Strip leading and trailing spaces
    .str.lower()  # Convert to lowercase for uniformity
)

# Debug: Check unique values
print("Unique counties in average sales price data:", average_sales_price_2023['county'].unique())
print("Unique names in geo_data:", geo_data['NAME'].unique())

# Debugging: Check counts of unique values
print("Number of unique counties in average sales price data:", len(average_sales_price_2023['county'].unique()))
print("Number of unique names in geo_data:", len(geo_data['NAME'].unique()))

# Check for counties in average sales price data that are not in geo_data
missing_in_geo_data = set(average_sales_price_2023['county']) - set(geo_data['NAME'])
print("Counties in average sales price data not found in geo_data:", missing_in_geo_data)

# Check for names in geo_data that are not in average sales price data
missing_in_sales_data = set(geo_data['NAME']) - set(average_sales_price_2023['county'])
print("Counties in geo_data not found in average sales price data:", missing_in_sales_data)

# Rename the sales price column for consistency if necessary
average_sales_price_2023.rename(columns={'YourActualPriceColumnName': 'Avg price'}, inplace=True)

# Merge GeoDataFrame with average sales price data
merged = geo_data.merge(average_sales_price_2023, left_on='NAME', right_on='county', how='left')

# Check for any rows without average sales data
no_sales_data = merged[merged['Avg price'].isnull()]
print("Merged DataFrame rows without sales data:", no_sales_data)

# Print merged DataFrame columns and sample data
print("Merged DataFrame columns:", merged.columns)
print("Merged DataFrame sample data:\n", merged.head())

# Function to determine color based on average price
def get_color(price):
    if price < 50000:
        return 'green'
    elif price < 100000:
        return 'yellow'
    elif price < 200000:
        return 'orange'
    elif price < 300000:
        return 'red'
    elif price < 400000:
        return 'purple'
    else:
        return 'darkblue'

# Create a map centered around Virginia
m = folium.Map(location=[37.4316, -78.6569], zoom_start=7)

# Add GeoJson with style based on average sales price
folium.GeoJson(
    merged,
    style_function=lambda feature: {
        'fillColor': get_color(feature['properties']['Avg price']),  # Ensure 'Avg price' exists
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(fields=['NAME', 'Avg price'])  # Display NAME and Avg price in tooltip
).add_to(m)

# Save the map
try:
    m.save("home_sales_prices.html")
    print("Map saved successfully!")
except Exception as e:
    print(f"Error saving map: {e}")


Unique counties in average sales price data: ['accomack' 'albemarle' 'alexandria' 'alleghany' 'amelia' 'amherst'
 'appomattox' 'arlington' 'augusta' 'bath' 'bedford' 'bland' 'botetourt'
 'bristol' 'brunswick' 'buchanan' 'buckingham' 'buena vista' 'campbell'
 'caroline' 'carroll' 'charles' 'charlotte' 'charlottesville' 'chesapeake'
 'chesterfield' 'clarke' 'colonial heights' 'covington' 'craig' 'culpeper'
 'cumberland' 'danville' 'dickenson' 'dinwiddie' 'emporia' 'essex'
 'fairfax' 'falls church' 'fauquier' 'floyd' 'fluvanna' 'franklin'
 'frederick' 'fredericksburg' 'galax' 'giles' 'gloucester' 'goochland'
 'grayson' 'greene' 'greensville' 'halifax' 'hampton' 'hanover'
 'harrisonburg' 'henrico' 'henry' 'highland' 'hopewell' 'isle of wight'
 'james' 'king and queen' 'king george' 'king william' 'lancaster' 'lee'
 'lexington' 'loudoun' 'louisa' 'lunenburg' 'lynchburg' 'madison'
 'manassas' 'manassas park' 'martinsville' 'mathews' 'mecklenburg'
 'middlesex' 'montgomery' 'nelson' 'new kent'

In [73]:
homes_owned_homes_rented = pd.read_excel("C:/Users/Tony/Downloads/owners_renters.xlsx")
print(homes_owned_homes_rented.columns)

Index(['county', 'total households', 'owner', 'tenant'], dtype='object')
